| SOURCE_TABLE_NAME | SOURCE_COLUMN_NAME | REPORTING_TABLE_NAME | REPORTING_COLUMN_NAME | TRANSFORMATION RULE | CONDITIONS |
| --- | --- |--- | --- |--- |--- |
| silver.daily_pricing_silver	| DATE_OF_PRICING	|reporting_fact_daily_pricing_gold| DATE_ID| Lookup Source DATE_OF_PRICING value against CALENDAR_DATE value on reporting_dim_date_gold table and select DATE_ID | Identify New/Changed Records From the Source |
| silver.daily_pricing_silver	| STATE_NAME	|reporting_fact_daily_pricing_gold| STATE_ID| Lookup Source STATE_NAME value against STATE_NAME value on reporting_dim_state_gold table and select STATE_ID |  |
| silver.daily_pricing_silver	| MARKET_NAME	|reporting_fact_daily_pricing_gold| MARKET_ID| Lookup Source MARKET_NAME value against MARKET_NAME value on reporting_dim_market_gold table and select MARKET_ID |  |
| silver.daily_pricing_silver	| PRODUCTGROUP_NAME , PRODUCT_NAME	|reporting_fact_daily_pricing_gold| PRODUCT_ID| Lookup Source PRODUCTGROUP_NAME and PRODUCT_NAME values against PRODUCTGROUP_NAME and PRODUCT_NAME values on reporting_dim_prduct_gold table and select PRODUCT_ID |  |
| silver.daily_pricing_silver	| VARIETY	|reporting_fact_daily_pricing_gold| VARIETY_ID| Lookup Source VARIETY value against VARIETY value on reporting_dim_variety_gold table and select VARIETY_ID |  |
| silver.daily_pricing_silver	| ROW_ID	|reporting_fact_daily_pricing_gold| ROW_ID| Direct Mapping |  |
| silver.daily_pricing_silver	| ARRIVAL_IN_TONNES	|reporting_fact_daily_pricing_gold| ARRIVAL_IN_TONNES| Direct Mapping |  |
| silver.daily_pricing_silver	| MINIMUM_PRICE	|reporting_fact_daily_pricing_gold| MINIMUM_PRICE| Direct Mapping |  |
| silver.daily_pricing_silver	| MAXIMUM_PRICE	|reporting_fact_daily_pricing_gold| MAXIMUM_PRICE| Direct Mapping |  |
| silver.daily_pricing_silver	| MODAL_PRICE	|reporting_fact_daily_pricing_gold| MODAL_PRICE| Direct Mapping |  |
| DERIVED	| DERIVED	|reporting_fact_daily_pricing_gold	| lakehouse_inserted_date	| Load current_timestamp() | |

In [0]:
%sql
INSERT INTO `pricing-analytics`.gold.reporting_fact_daily_pricing_gold
SELECT
dateDimGold.DATE_ID
,stateDimGold.STATE_ID
,marketDimGold.MARKET_ID
,productDimGold.PRODUCT_ID
,varietyDimGold.VARIETY_ID
,pricingSilverFact.ROW_ID
,pricingSilverFact.ARRIVAL_IN_TONNES
,pricingSilverFact.MAXIMUM_PRICE
,pricingSilverFact.MINIMUM_PRICE
,pricingSilverFact.MODAL_PRICE
,current_timestamp()
,current_timestamp()
FROM `pricing-analytics`.silver.daily_pricing_silver pricingSilverFact
LEFT OUTER JOIN `pricing-analytics`.gold.reporting_dim_date_gold dateDimGold
on pricingSilverFact.DATE_OF_PRICING = dateDimGold.CALENDAR_DATE
LEFT OUTER JOIN `pricing-analytics`.gold.reporting_dim_state_gold stateDimGold
on pricingSilverFact.STATE_NAME = stateDimGold.STATE_NAME
LEFT OUTER JOIN `pricing-analytics`.gold.reporting_dim_market_gold marketDimGold
on pricingSilverFact.MARKET_NAME = marketDimGold.MARKET_NAME
LEFT OUTER JOIN `pricing-analytics`.gold.reporting_dim_product_gold productDimGold
on pricingSilverFact.PRODUCT_NAME = productDimGold.PRODUCT_NAME
AND pricingSilverFact.PRODUCTGROUP_NAME = productDimGold.PRODUCTGROUP_NAME
LEFT OUTER JOIN `pricing-analytics`.gold.reporting_dim_variety_gold varietyDimGold
on pricingSilverFact.VARIETY = varietyDimGold.VARIETY
WHERE pricingSilverFact.lakehouse_updated_date > (SELECT nvl(max(PROCESSED_TABLE_DATETIME),'1900-01-01') FROM `pricing-analytics`.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
WHERE process_name = 'reportingFactTableLoad' AND process_status = 'Completed' )

In [0]:
%sql
INSERT INTO  `pricing-analytics`.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(PROCESS_NAME,PROCESSED_TABLE_DATETIME,PROCESS_STATUS)
SELECT 'reportingFactTableLoad' , max(lakehouse_updated_date) ,'Completed' FROM `pricing-analytics`.silver.daily_pricing_silver